In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1582306697164_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1582306697164_0002,pyspark,idle,Link,Link,
2,application_1582306697164_0003,pyspark,idle,Link,Link,✔


In [3]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.1-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [4]:
import pandas as pd
import re
from pyspark.sql.functions import col, udf


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

def run_sql_pandas(statement):
    return run_sql(statement).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
clean_sdf = spark.read.json("s3://topic-sentiment-1/clean-json-data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
clean_sdf.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- language: string (nullable = true)
 |-- language_guess: string (nullable = true)
 |-- month: long (nullable = true)
 |-- published: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- text_or_desc: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- year: long (nullable = true)

In [8]:
run_sql('drop database if exists topic cascade')
run_sql('create database topic')
permanent_table_name = 'topic.articles'
clean_sdf.write.format("parquet").saveAsTable(permanent_table_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
by_domain = run_sql_pandas('''
    SELECT source_domain, count(*) AS article_count
    FROM topic.articles
    GROUP BY source_domain
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
by_domain.head(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             source_domain  article_count
0                   ft.com          23424
1              nytimes.com          33009
2     chicago.suntimes.com          31995
3             theblaze.com          42681
4              reuters.com          53082
5                msnbc.com            922
6   washingtonexaminer.com          65171
7          nydailynews.com          62728
8      americanthinker.com          84987
9          theatlantic.com          69096
10             newsmax.com          23852
11             nbcnews.com          60891
12      washingtonpost.com           3542
13                 wsj.com           9134
14       foreignpolicy.com          30336
15               slate.com          74673
16              boston.com          22804
17           economist.com          26567
18             latimes.com          90510
19         newrepublic.com          65405
20                 ajc.com           8135
21           thenation.com          39497
22                 bbc.com        

In [11]:
duplicates = run_sql_pandas('''
    SELECT text_or_desc, count(*) AS duplicate_count
    FROM topic.articles
    GROUP BY text_or_desc
    HAVING count(*) > 1
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
duplicates.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                         text_or_desc  duplicate_count
0                                            Holidays               26
1   Credit... The New York Times Archives\nFebruar...                4
2   RT France reporter Lucas Leger says he was hit...                2
3   The Battleground Poll, the most respected and ...                2
4   Press release content from Wired Release. The ...                2
..                                                ...              ...
95  Israel began the evacuation of an illegal West...                2
96                                   State Department                7
97  America's 2019 defense budget has suspended US...                2
98  The regional parliament in Catalonia approved ...                2
99  First off, why exactly was Paula sitting on Si...                2

[100 rows x 2 columns]

In [13]:
clean_sdf.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1989813

In [14]:
duplicates.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

text_or_desc       92572
duplicate_count    92572
dtype: int64

In [15]:
by_urls = run_sql_pandas('''
    SELECT url, count(*) AS duplicate_count
    FROM topic.articles
    GROUP BY url
    HAVING count(*) > 1
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
by_urls.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                  url  duplicate_count
0   https://projects.newsday.com/schools/school/br...                3
1   https://www.rt.com/news/212059-steve-jobs-post...                2
2   https://www.rt.com/russia/431481-kadyrov-found...                2
3   https://www.rt.com/uk/453838-brexit-delay-a50-...                2
4   https://www.rt.com/uk/201511-uk-foreign-office...                2
..                                                ...              ...
95  https://www.rt.com/news/439059-trump-delays-ru...                2
96  https://www.rt.com/news/454746-israel-invasion...                2
97  https://www.rt.com/uk/209071-man-passport-citi...                2
98  https://www.rt.com/news/428282-chinese-police-...                2
99  https://www.rt.com/news/455756-dutch-f16-shot-...                2

[100 rows x 2 columns]

In [17]:
by_urls.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

url                39425
duplicate_count    39425
dtype: int64

In [18]:
clean_sdf = clean_sdf.dropDuplicates(['url'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# now recount the number of text duplicates
duplicates = run_sql_pandas('''
    SELECT text_or_desc, count(*) AS duplicate_count
    FROM topic.articles
    GROUP BY text_or_desc
    HAVING count(*) > 1
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
duplicates.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

text_or_desc       92572
duplicate_count    92572
dtype: int64

In [21]:
clean_sdf.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1947932

In [22]:
run_sql('''
    REFRESH TABLE topic.articles
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [23]:
# now recount the number of text duplicates
duplicates = run_sql_pandas('''
    SELECT text_or_desc, count(*) AS duplicate_count
    FROM topic.articles
    GROUP BY text_or_desc
    HAVING count(*) > 1
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
duplicates.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

text_or_desc       92572
duplicate_count    92572
dtype: int64

In [25]:
clean_sdf = clean_sdf.dropDuplicates(['text_or_desc'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
clean_sdf.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1796896

In [27]:
run_sql('''
    REFRESH TABLE topic.articles
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [28]:
# now recount the number of text duplicates
duplicates = run_sql_pandas('''
    SELECT text_or_desc, count(*) AS duplicate_count
    FROM topic.articles
    GROUP BY text_or_desc
    HAVING count(*) > 1
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
duplicates.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                         text_or_desc  duplicate_count
0                                            Holidays               26
1   Credit... The New York Times Archives\nFebruar...                4
2   RT France reporter Lucas Leger says he was hit...                2
3   The Battleground Poll, the most respected and ...                2
4   Press release content from Wired Release. The ...                2
..                                                ...              ...
95  Israel began the evacuation of an illegal West...                2
96                                   State Department                7
97  America's 2019 defense budget has suspended US...                2
98  The regional parliament in Catalonia approved ...                2
99  First off, why exactly was Paula sitting on Si...                2

[100 rows x 2 columns]

In [30]:
duplicates.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

text_or_desc       92572
duplicate_count    92572
dtype: int64

**So I guess that refreshing the table does nothing event when the dataframe has changed**

In [31]:
permanent_table_name = 'topic.articles_after'
clean_sdf.write.format("parquet").saveAsTable(permanent_table_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# now recount the number of text duplicates
duplicates = run_sql_pandas('''
    SELECT text_or_desc, count(*) AS duplicate_count
    FROM topic.articles_after
    GROUP BY text_or_desc
    HAVING count(*) > 1
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
duplicates.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

text_or_desc       0
duplicate_count    0
dtype: int64

In [34]:
clean_sdf.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1796896